<!-- # Copyright (c) 2025 takotime808 -->

### Report Generation

In [1]:
!cd .. && python multioutreg/report/generate_report1.py


Bad key text.markup in file /Users/tako/.matplotlib/matplotlibrc, line 165 ("text.markup         : 'plain'  # Affects how text, such as titles and labels, are")
You probably need to get an updated matplotlibrc file from
https://github.com/matplotlib/matplotlib/blob/v3.6.3/matplotlibrc.template
or from the matplotlib source distribution
/Users/tako/work/git/account_takotime808/multioutreg/multioutreg/report/generate_report1.py:65: FutureWarning: The NumPy global RNG was seeded by calling `np.random.seed`. In a future version this function will no longer use the global RNG. Pass `rng` explicitly to opt-in to the new behaviour and silence this warning.
  shap.summary_plot(shap_values, X, show=False)
Report written to outputs/surrogate_report_example.html


In [2]:
!cd .. && python multioutreg/report/generate_report1_safe.py


Bad key text.markup in file /Users/tako/.matplotlib/matplotlibrc, line 165 ("text.markup         : 'plain'  # Affects how text, such as titles and labels, are")
You probably need to get an updated matplotlibrc file from
https://github.com/matplotlib/matplotlib/blob/v3.6.3/matplotlibrc.template
or from the matplotlib source distribution
/Users/tako/work/git/account_takotime808/multioutreg/multioutreg/report/generate_report1_safe.py:63: FutureWarning: The NumPy global RNG was seeded by calling `np.random.seed`. In a future version this function will no longer use the global RNG. Pass `rng` explicitly to opt-in to the new behaviour and silence this warning.
  shap.summary_plot(shap_values, X, show=False)
Report written to outputs/surrogate_report_example_safe_fig.html


In [3]:
# !cd .. && python examples/generate_report.py
!cd .. && python multioutreg/report/generate_report.py


Bad key text.markup in file /Users/tako/.matplotlib/matplotlibrc, line 165 ("text.markup         : 'plain'  # Affects how text, such as titles and labels, are")
You probably need to get an updated matplotlibrc file from
https://github.com/matplotlib/matplotlib/blob/v3.6.3/matplotlibrc.template
or from the matplotlib source distribution
/Users/tako/codes/miniconda3/envs/foo310/lib/python3.10/site-packages/multioutreg/figures/prediction_plots.py:115: UserWarning: Matplotlib is currently using agg, which is a non-GUI backend, so cannot show the figure.
  plt.show()
/Users/tako/codes/miniconda3/envs/foo310/lib/python3.10/site-packages/multioutreg/figures/residuals.py:116: UserWarning: Matplotlib is currently using agg, which is a non-GUI backend, so cannot show the figure.
  plt.show()
/Users/tako/codes/miniconda3/envs/foo310/lib/python3.10/site-packages/multioutreg/figures/coverage_plots.py:72: UserWarning: Matplotlib is currently using agg, which is a non-GUI backend, so cannot show the

In [4]:
# from jinja2 import Environment, FileSystemLoader

# env = Environment(loader=FileSystemLoader('multioutreg/report/template.html'))
# template = env.get_template('template.html')

# report_html = template.render(
#     project_title="Your Model Report",
#     model_type="Multi-fidelity Gaussian Process",
#     fidelity_levels=['Low', 'High'],
#     output_names=['Y1', 'Y2'],
#     description="Explains a multi-fidelity surrogate using X, Y...",
#     metrics={'Y1': {'r2': 0.92, 'rmse': 0.2, 'mae': 0.15}, 'Y2': {...}},
#     uncertainty_metrics={'NLL': 1.23, 'Sharpness': 0.45, ...},
#     uncertainty_plots=[{'img_b64': ..., 'title': "Calibration Curve", 'caption': "..."}],
#     prediction_plots={'Y1': ..., 'Y2': ...},
#     shap_plots={'Y1': ..., 'Y2': ...},
#     pdp_plots={'Y1': ..., 'Y2': ...},
#     sampling_umap_plot=...,
#     sampling_method_explanation="Sampling appears to be LHS, justified by...",
#     sampling_other_plots=[...],
#     other_plots=[...],
#     n_train=100,
#     n_test=40,
#     cross_validation="5-fold",
#     seed=42,
#     notes="Any extra info\nAnother note"
# )
